In [2]:
from selenium import webdriver

from pymongo import MongoClient
import re
import pandas as pd
from bs4 import BeautifulSoup
import json

In [3]:
player = {
    'id': 1629121,
    'stats': {}
}
browser = webdriver.Chrome(executable_path="./chromedriver")
browser.get(f'https://stats.nba.com/player/{player["id"]}/?SeasonType=Regular%20Season')
# Get player name and format it for the folder name
name = browser.find_elements_by_xpath('//div[@class="player-summary__player-name"]')
position = browser.find_elements_by_xpath('//span[@ng-if="playerInfo.POSITION_INITIALS"]')
player['name'] = ' '.join(name[0].text.split('\n'))
player['position'] = position[0].text if len(position) > 0 else ''
# Find the table where all the statistics tables are
stats_tables = browser.find_elements_by_xpath('//nba-stat-table')
# Iterate over each table and create your own csv file
for index_table, stats_table in enumerate(stats_tables):
    # Get the table name
    caption_item = browser.find_elements_by_xpath('.//div[@class="nba-stat-table__caption"]')
    caption = 'Table'+str(index_table) if len(caption_item) == 0 else caption_item[index_table].text
    player['stats'][caption] = []
    # Get the columns names
    head = []
    col_list = stats_table.find_elements_by_xpath('.//div[@class="nba-stat-table__overflow"]/table/thead/tr/th')
    # Get the values of each row
    row_list = stats_table.find_elements_by_xpath('.//div[@class="nba-stat-table__overflow"]/table/tbody/tr')
    for index_row, row in enumerate(row_list):
        body = {}
        body_col_list = row.find_elements_by_xpath('.//td')
        for index_col, row_col in enumerate(body_col_list):
            body[col_list[index_col].text] = row_col.text
        player['stats'][caption].append(body)
print(player)
browser.close()

{'id': 1629121, 'stats': {'Traditional Splits': [{'BY YEAR': '2018-19', 'TEAM': 'ATL', 'GP': '34', 'MIN': '12.6', 'PTS': '3.2', 'FGM': '1.1', 'FGA': '3.2', 'FG%': '34.5', '3PM': '0.7', '3PA': '2.2', '3P%': '33.8', 'FTM': '0.2', 'FTA': '0.3', 'FT%': '77.8', 'OREB': '0.3', 'DREB': '1.4', 'REB': '1.8', 'AST': '1.9', 'TOV': '0.8', 'STL': '0.4', 'BLK': '0.1', 'PF': '1.3', 'FP': '9.0', 'DD2': '0', 'TD3': '0', '+/-': '-1.7'}], 'Advanced Splits': [{'BY YEAR': '2018-19', 'TEAM': 'ATL', 'GP': '34', 'MIN': '12.6', 'OFFRTG': '102.5', 'DEFRTG': '109.1', 'NETRTG': '-6.5', 'AST%': '20.1', 'AST/TO': '2.32', 'AST RATIO': '31.6', 'OREB%': '2.2', 'DREB%': '10.9', 'REB%': '6.3', 'TO RATIO': '13.6', 'EFG%': '45.9', 'TS%': '47.4', 'USG%': '13.1', 'PACE': '106.81', 'PIE': '5.5'}], 'Misc Splits': [{'BY YEAR': '2018-19', 'TEAM': 'ATL', 'GP': '34', 'MIN': '12.6', 'PTS OFF TO': '0.6', '2ND PTS': '0.2', 'FBPS': '0.3', 'PITP': '0.5', 'OPP\nPTS OFF TO': '4.7', 'OPP\n2ND PTS': '4.0', 'OPP\nFBPS': '3.2', 'OPP\nPITP':

In [4]:
player = {
    'id': 1629121,
    'stats': {}
}
browser = webdriver.Chrome(executable_path="./chromedriver")
browser.get(f'https://stats.nba.com/player/{player["id"]}/?SeasonType=Regular%20Season')
# Get player name and format it for the folder name
name = browser.find_elements_by_xpath('//div[@class="player-summary__player-name"]')
position = browser.find_elements_by_xpath('//span[@ng-if="playerInfo.POSITION_INITIALS"]')
player['name'] = ' '.join(name[0].text.split('\n'))
player['position'] = position[0].text if len(position) > 0 else ''

caption_item = browser.find_elements_by_xpath('.//div[@class="nba-stat-table__caption"]')

soup = BeautifulSoup(browser.page_source, 'lxml')
tables = soup.findAll('nba-stat-table')

for index, table in enumerate(tables):
    df = pd.read_html(table.prettify())

    print(caption_item[index].text)
    print(df[0].head())
    
    json_stats = df[0].to_dict(orient='records')
    
    print(json_stats)
browser.close()

Traditional Splits
   By Year TEAM  GP   MIN  PTS  FGM  FGA   FG%  3PM  3PA  ...  REB  AST  TOV  \
0  2018-19  ATL  34  12.6  3.2  1.1  3.2  34.5  0.7  2.2  ...  1.8  1.9  0.8   

   STL  BLK   PF   FP  DD2  TD3  +/-  
0  0.4  0.1  1.3  9.0    0    0 -1.7  

[1 rows x 26 columns]
[{'By Year': '2018-19', 'TEAM': 'ATL', 'GP': 34, 'MIN': 12.6, 'PTS': 3.2, 'FGM': 1.1, 'FGA': 3.2, 'FG%': 34.5, '3PM': 0.7, '3PA': 2.2, '3P%': 33.8, 'FTM': 0.2, 'FTA': 0.3, 'FT%': 77.8, 'OREB': 0.3, 'DREB': 1.4, 'REB': 1.8, 'AST': 1.9, 'TOV': 0.8, 'STL': 0.4, 'BLK': 0.1, 'PF': 1.3, 'FP': 9.0, 'DD2': 0, 'TD3': 0, '+/-': -1.7}]
Advanced Splits
   By Year TEAM  GP   MIN  OffRtg  DefRtg  NetRtg  AST%  AST/TO  AST Ratio  \
0  2018-19  ATL  34  12.6   102.5   109.1    -6.5  20.1    2.32       31.6   

   OREB%  DREB%  REB%  TO Ratio  eFG%   TS%  USG%    PACE  PIE  
0    2.2   10.9   6.3      13.6  45.9  47.4  13.1  106.81  5.5  
[{'By Year': '2018-19', 'TEAM': 'ATL', 'GP': 34, 'MIN': 12.6, 'OffRtg': 102.5, 'DefRtg': 